In [10]:
# Stop warnings
import warnings
warnings.filterwarnings("ignore")

# Debug
import ipdb
deb = ipdb.set_trace

# General imports
import os
import sys
import json
import datetime
import numpy as np

# MRI analysis imports
import nibabel as nb
from prfpy.stimulus import PRFStimulus2D
from prfpy.model import Iso2DGaussianModel, CSS_Iso2DGaussianModel
from prfpy.fit import Iso2DGaussianFitter, CSS_Iso2DGaussianFitter

# Personal imports
sys.path.append("{}/../../analysis_code/utils".format(os.getcwd()))
from surface_utils import load_surface ,make_surface_image
from pycortex_utils import data_from_rois, set_pycortex_config_file
from maths_utils import r2_score_surf
from screen_utils import get_screen_settings

In [12]:
# Get inputs
start_time = datetime.datetime.now()
    
# Inputs
main_dir = '/Users/uriel/disks/meso_shared'
project_dir = 'RetinoMaps'
subject = 'sub-02'
sub_num = subject[4:]
input_fn = '{}/{}/derivatives/pp_data/{}/fsnative/func/fmriprep_dct_loo_avg/{}_task-pRF_hemi-L_fmriprep_dct_avg_loo-4_bold.func.gii'.format(main_dir, project_dir, subject, subject)
n_jobs = int(32)
output_folder = "prf"

n_batches = n_jobs
verbose = True
css_params_num = 9

In [70]:
# Analysis parameters
with open('../settings.json') as f:
    json_s = f.read()
    analysis_info = json.loads(json_s)

# # Load screen settings from subject dependend task-events.json
# screen_size_cm, screen_distance_cm = get_screen_settings(main_dir,project_dir, sub_num, analysis_info['prf_task_name'])

screen_size_cm = analysis_info['screen_size_cm'] 
screen_distance_cm = analysis_info['screen_distance_cm'] 
vdm_width = analysis_info['vdm_size_pix'][0] 
vdm_height = analysis_info['vdm_size_pix'][1]
TR = analysis_info['TR']
gauss_grid_nr = analysis_info['gauss_grid_nr']
max_ecc_size = analysis_info['max_ecc_size']
n_th = analysis_info['n_th']
rsq_iterative_th = analysis_info['rsq_iterative_th']
css_grid_nr = analysis_info['css_grid_nr']
prf_task_name = analysis_info['prf_task_name']
ecc_th = analysis_info['ecc_th']
size_th = analysis_info['size_th']

# Set pycortex db and colormaps
cortex_dir = "{}/{}/derivatives/pp_data/cortex".format(main_dir, project_dir)
set_pycortex_config_file(cortex_dir)

In [68]:
analysis_info.keys()

dict_keys(['project_name', 'account', 'webapp_login', 'webapp_dir', 'sessions', 'subjects', 'anat_session', 'func_session', 'outlier_prf_beh', 'task_names', 'prf_task_name', 'task_glm', 'task_intertask', 'cluster_name', 'TR', 'TRs', 'num_steps_kappa_pRF', 'fit_rsq_threshold', 'high_pass_threshold', 'formats', 'extensions', 'n_runs_prf', 'screen_size_pix', 'screen_size_cm', 'screen_distance_cm', 'vdm_size_pix', 'vdm_size_pix_eyes_mvt', 'screen_width', 'screen_height', 'screen_width_px', 'screen_height_px', 'eye_mov_height_dva', 'pix_ratio', 'gauss_grid_nr', 'css_grid_nr', 'max_ecc_size', 'stim_width', 'stim_height', 'rois', 'mmp_rois', 'rois_groups', 'ecc_th', 'size_th', 'rsqr_th', 'n_th', 'amplitude_th', 'stats_th', 'alpha_range', 'rsq_iterative_th', 'glm_alpha', 'fdr_alpha', 'glm_confounds', 'vertex_pcm_rad', 'maps_names_corr', 'maps_names_gauss', 'maps_names_css', 'maps_names_pcm', 'maps_names_css_stats', 'glm_code_names', 'maps_names_intertask', 'correlation_type'])

In [16]:
# Define directories and files names (fn)
if input_fn.endswith('.nii'):
    prf_fit_dir = "{}/{}/derivatives/pp_data/{}/170k/{}/fit".format(
        main_dir, project_dir, subject, output_folder)
    os.makedirs(prf_fit_dir, exist_ok=True)
    rois = analysis_info['mmp_rois']

elif input_fn.endswith('.gii'):
    prf_fit_dir = "{}/{}/derivatives/pp_data/{}/fsnative/{}/fit".format(
        main_dir, project_dir, subject, output_folder)
    os.makedirs(prf_fit_dir, exist_ok=True)
    rois = analysis_info['rois']

fit_fn_css = input_fn.split('/')[-1]
fit_fn_css = fit_fn_css.replace('bold', 'prf-fit_css')

pred_fn_css = input_fn.split('/')[-1] 
pred_fn_css = pred_fn_css.replace('bold', 'prf-pred_css')

# Get task specific visual design matrix
vdm_fn = '{}/{}/derivatives/vdm/vdm_{}_{}_{}.npy'.format(
    main_dir, project_dir, prf_task_name, vdm_width, vdm_height)
vdm = np.load(vdm_fn)

# Define model parameter grid range
sizes = max_ecc_size * np.linspace(0.1, 1, gauss_grid_nr)**2
eccs = max_ecc_size * np.linspace(0.1, 1, gauss_grid_nr)**2
polars = np.linspace(0, 2 * np.pi, gauss_grid_nr)
exponent_css_grid = np.linspace(n_th[0], n_th[1], css_grid_nr)

print("\n===== PRF FIT PARAMETERS =====")
print(f"Screen Size (cm): {screen_size_cm}")
print(f"Screen Distance (cm): {screen_distance_cm}")
print(f"TR: {TR}")
print(f"Max eccentricity/size values: {max_ecc_size}")
print(f"CSS exponentgrid: {exponent_css_grid}")
print("==============================\n")


===== PRF FIT PARAMETERS =====
Screen Size (cm): [77.3, 43.5]
Screen Distance (cm): 120
TR: 1.2
Max eccentricity/size values: 20
CSS exponentgrid: [0.01       0.03828571 0.06657143 0.09485714 0.12314286 0.15142857
 0.17971429 0.208      0.23628571 0.26457143 0.29285714 0.32114286
 0.34942857 0.37771429 0.406      0.43428571 0.46257143 0.49085714
 0.51914286 0.54742857 0.57571429 0.604      0.63228571 0.66057143
 0.68885714 0.71714286 0.74542857 0.77371429 0.802      0.83028571
 0.85857143 0.88685714 0.91514286 0.94342857 0.97171429 1.        ]



In [18]:
# Load data
img, data, data_roi, roi_idx = data_from_rois(fn=input_fn, 
                                              subject=subject, 
                                              rois=rois)
print('roi extraction done')

roi extraction done


In [8]:
data_roi[].shape

SyntaxError: invalid syntax (2266457944.py, line 1)

In [19]:
# Determine visual design
stimulus = PRFStimulus2D(screen_size_cm=screen_size_cm[1],
                         screen_distance_cm=screen_distance_cm,
                         design_matrix=vdm, 
                         TR=TR)

print("\n===== PRF MODEL PARAMETERS =====")
print("Stimulus x min/max (deg):", np.nanmin(stimulus.x_coordinates ), np.nanmax(stimulus.x_coordinates ))
print("Stimulus y min/max (deg) :", np.nanmin(stimulus.y_coordinates), np.nanmax(stimulus.y_coordinates))
print("Eccentricity grid range:", np.min(eccs), np.max(eccs))
print("Eccentricity grid range:", np.min(sizes), np.max(sizes))
print("==============================\n")


===== PRF MODEL PARAMETERS =====
Stimulus x min/max (deg): -10.273330641629286 10.273330641629286
Stimulus y min/max (deg) : -10.273330641629286 10.273330641629286
Eccentricity grid range: 0.20000000000000004 20.0
Eccentricity grid range: 0.20000000000000004 20.0



In [20]:
# Gauss fit
# ---------

# Define gauss model
gauss_model = Iso2DGaussianModel(stimulus=stimulus)

# Grid fit gauss model
gauss_fitter = Iso2DGaussianFitter(data=data_roi[:,:10].T, 
                                   model=gauss_model, 
                                   n_jobs=n_jobs)

gauss_fitter.grid_fit(ecc_grid=eccs, 
                      polar_grid=polars, 
                      size_grid=sizes, 
                      verbose=verbose, 
                      n_batches=n_batches)

# Iterative fit gauss model
gauss_fitter.iterative_fit(rsq_threshold=rsq_iterative_th, verbose=verbose)
gauss_fit = gauss_fitter.iterative_search_params

Each batch contains approx. 1 voxels.


[Parallel(n_jobs=32)]: Using backend LokyBackend with 32 concurrent workers.
/Users/uriel/softwares/anaconda3/envs/prf_env/prfpy/prfpy/fit.py:441: RuntimeWarning: divide by zero encountered in divide
  slopes = (n_timepoints * np.dot(vox_data, predictions.T) - sumd *
/Users/uriel/softwares/anaconda3/envs/prf_env/prfpy/prfpy/fit.py:441: RuntimeWarning: invalid value encountered in divide
  slopes = (n_timepoints * np.dot(vox_data, predictions.T) - sumd *
/Users/uriel/softwares/anaconda3/envs/prf_env/prfpy/prfpy/fit.py:441: RuntimeWarning: divide by zero encountered in divide
  slopes = (n_timepoints * np.dot(vox_data, predictions.T) - sumd *
/Users/uriel/softwares/anaconda3/envs/prf_env/prfpy/prfpy/fit.py:441: RuntimeWarning: invalid value encountered in divide
  slopes = (n_timepoints * np.dot(vox_data, predictions.T) - sumd *
/Users/uriel/softwares/anaconda3/envs/prf_env/prfpy/prfpy/fit.py:441: RuntimeWarning: divide by zero encountered in divide
  slopes = (n_timepoints * np.dot(vox_

In [72]:
# CSS fit
# -------

# bounds = [
#     (np.nanmin(stimulus.x_coordinates ), np.nanmax(stimulus.x_coordinates )),       # x
#     (np.nanmin(stimulus.y_coordinates ), np.nanmax(stimulus.y_coordinates )),       # y
#     (0.05, max_ecc_size),     # sigma
#     (n_th[0], n_th[1]),    # n
#     (0, 1000),      # amplitude
#     (0, 0)      # baseline
# ]

# css_fitter.iterative_fit(
#     rsq_threshold=rsq_iterative_th,
#     verbose=True,
#     xtol=1e-4,
#     ftol=1e-4
# )

from prfpy.fit import CSS_Iso2DGaussianFitter

# définir bornes [min, max] pour chaque paramètre du modèle CSS
# ordre des paramètres optimisés pour CSS
# [mu_x, mu_y, size, beta, baseline, n, hrf_1, hrf_2]


bounds =  [(np.nanmin(stimulus.x_coordinates ), np.nanmax(stimulus.x_coordinates )),  # x
                (np.nanmin(stimulus.y_coordinates ), np.nanmax(stimulus.y_coordinates )),  # y
                (size_th[0], size_th[1]),  # prf size
                (0, 1000),  # prf amplitude
                (0, 0),  # bold baseline (fixed to zero)
                (n_th[0], n_th[1]),  #n
                (-np.inf, np.inf),  # hrf1
                (-np.inf, np.inf) # hrf1
          ] 



# # Define CSS model
# css_model = CSS_Iso2DGaussianModel(stimulus=stimulus)

# # Grid fit CSS model
# css_fitter = CSS_Iso2DGaussianFitter(data=data_roi[:,:10].T, 
#                                      model=css_model, 
#                                      n_jobs=n_jobs,
#                                      use_previous_gaussian_fitter_hrf=False,
#                                      previous_gaussian_fitter=gauss_fitter)
# # Run css grid fit
# css_fitter.grid_fit(exponent_grid=exponent_css_grid,
#                     verbose=verbose,
#                     n_batches=n_batches)

# # # Run iterative fit
# # css_fitter.iterative_fit(rsq_threshold=rsq_iterative_th, 
# #                          verbose=verbose, 
# #                          xtol=1e-4, 
# #                          ftol=1e-4)


css_fitter.iterative_fit(rsq_threshold=rsq_iterative_th,
                         verbose=verbose,
                         xtol=1e-4,
                         ftol=1e-4,
                         bounds=bounds)


# css_fit = css_fitter.iterative_search_params

# # Rearrange result of CSS model
# css_fit_mat = np.full((data.shape[1], css_params_num), np.nan, dtype=float)
# css_pred_mat = np.full_like(data, np.nan, dtype=float) 

# for est, vert in enumerate(roi_idx):
#     css_fit_mat[vert] = css_fit[est]
#     css_pred_mat[:,vert] = css_model.return_prediction(mu_x=css_fit[est][0],
#                                                       mu_y=css_fit[est][1], 
#                                                       size=css_fit[est][2], 
#                                                       beta=css_fit[est][3], 
#                                                       baseline=css_fit[est][4],
#                                                       n=css_fit[est][5],
#                                                       hrf_1=css_fit[est][6],
#                                                       hrf_2=css_fit[est][7])
        
# # Compute loo r2
# loo_bold_fn = input_fn.replace('dct_avg_loo', 'dct_loo')
# loo_img, loo_bold = load_surface(fn=loo_bold_fn)
# loo_r2 = r2_score_surf(bold_signal=loo_bold, model_prediction=css_pred_mat)

# # Add loo r2 css_fit_mat
# css_fit_mat = np.column_stack((css_fit_mat, loo_r2))

# # Export data
# maps_names = ['mu_x', 'mu_y', 'prf_size', 'prf_amplitude', 'bold_baseline',
#               'n', 'hrf_1','hrf_2', 'r_squared','loo_r2']

[Parallel(n_jobs=32)]: Using backend LokyBackend with 32 concurrent workers.


RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            7     M =           10

At X0         1 variables are exactly at the bounds

At iterate    0    f=  6.80403D+01    |proj g|=  6.76877D+00

At iterate    1    f=  6.79065D+01    |proj g|=  9.99991D+02

At iterate    2    f=  6.53736D+01    |proj g|=  9.99990D+02

At iterate    3    f=  6.36883D+01    |proj g|=  1.64660D+02

At iterate    4    f=  6.36363D+01    |proj g|=  2.49477D+00

At iterate    5    f=  6.36226D+01    |proj g|=  1.15440D+00

At iterate    6    f=  6.35636D+01    |proj g|=  1.52049D+02

At iterate    7    f=  6.35171D+01    |proj g|=  3.07246D+02
RUNNING THE L-BFGS-B CODE

           * * *

Machine precision = 2.220D-16
 N =            7     M =           10

At X0         1 variables are exactly at the bounds

At iterate    0    f=  7.17089D+01    |proj g|=  3.88854D+00

At iterate    1    f=  6.54811D+01    |proj g|=  5.10607D+02

At iterate    8    f=  6.34402D+01    |proj g

[Parallel(n_jobs=32)]: Done   2 out of  10 | elapsed:    1.9s remaining:    7.7s

   evaluations in the last line search.  Termination
   may possibly be caused by a bad search direction.

   evaluations in the last line search.  Termination
   may possibly be caused by a bad search direction.

   evaluations in the last line search.  Termination
   may possibly be caused by a bad search direction.

   evaluations in the last line search.  Termination
   may possibly be caused by a bad search direction.
[Parallel(n_jobs=32)]: Done  10 out of  10 | elapsed:    2.0s finished


In [46]:
bounds_upper

[10.273330641629286, 10.273330641629286, 20, 1000, 1000, 1, inf, inf, 1]

In [ ]:
# # Export fit
# img_css_fit_mat = make_surface_image(data=css_fit_mat.T, 
#                                      source_img=img, 
#                                      maps_names=maps_names)

# nb.save(img_css_fit_mat,'{}/{}'.format(prf_fit_dir, fit_fn_css)) 

# # Export prediction
# img_css_pred_mat = make_surface_image(data=css_pred_mat, source_img=img)
# nb.save(img_css_pred_mat,'{}/{}'.format(prf_fit_dir, pred_fn_css)) 

# # Print duration
# end_time = datetime.datetime.now()
# print("\nStart time:\t{start_time}\nEnd time:\t{end_time}\nDuration:\t{dur}".format(start_time=start_time, 
#                                                                                     end_time=end_time, 
#                                                                                     dur=end_time - start_time))

Each batch contains approx. 1 voxels.


[Parallel(n_jobs=32)]: Using backend LokyBackend with 32 concurrent workers.
[Parallel(n_jobs=32)]: Done   1 tasks      | elapsed:    0.1s
[Parallel(n_jobs=32)]: Batch computation too fast (0.05176091194152832s.) Setting batch_size=2.
[Parallel(n_jobs=32)]: Done   2 out of  32 | elapsed:    0.1s remaining:    0.9s
[Parallel(n_jobs=32)]: Done   5 out of  32 | elapsed:    0.1s remaining:    0.4s
[Parallel(n_jobs=32)]: Done   8 out of  32 | elapsed:    0.1s remaining:    0.3s
[Parallel(n_jobs=32)]: Done  11 out of  32 | elapsed:    0.1s remaining:    0.2s
[Parallel(n_jobs=32)]: Done  14 out of  32 | elapsed:    0.1s remaining:    0.2s
[Parallel(n_jobs=32)]: Done  17 out of  32 | elapsed:    0.3s remaining:    0.2s
[Parallel(n_jobs=32)]: Done  20 out of  32 | elapsed:    2.3s remaining:    1.4s
[Parallel(n_jobs=32)]: Done  23 out of  32 | elapsed:    2.3s remaining:    0.9s
[Parallel(n_jobs=32)]: Done  26 out of  32 | elapsed:    2.3s remaining:    0.5s
[Parallel(n_jobs=32)]: Done  29 out 

Performing unbounded, unconstrained minimization (Powell).
Performing unbounded, unconstrained minimization (Powell).
Performing unbounded, unconstrained minimization (Powell).
Performing unbounded, unconstrained minimization (Powell).
Performing unbounded, unconstrained minimization (Powell).
Performing unbounded, unconstrained minimization (Powell).
Performing unbounded, unconstrained minimization (Powell).
Optimization terminated successfully.
         Current function value: 51.005389
         Iterations: 4
         Function evaluations: 327
Optimization terminated successfully.
         Current function value: 48.342517
         Iterations: 4
         Function evaluations: 323
Optimization terminated successfully.
         Current function value: 49.993438
         Iterations: 4
         Function evaluations: 330
Optimization terminated successfully.
         Current function value: 46.590624
         Iterations: 6
         Function evaluations: 483
Optimization terminated success

[Parallel(n_jobs=32)]: Done   2 out of  10 | elapsed:    0.3s remaining:    1.2s


Performing unbounded, unconstrained minimization (Powell).
Performing unbounded, unconstrained minimization (Powell).
Performing unbounded, unconstrained minimization (Powell).
Optimization terminated successfully.
         Current function value: 46.829717
         Iterations: 4
         Function evaluations: 326
Optimization terminated successfully.
         Current function value: 48.577089
         Iterations: 6
         Function evaluations: 500
Optimization terminated successfully.
         Current function value: 44.356650
         Iterations: 6
         Function evaluations: 512


[Parallel(n_jobs=32)]: Done  10 out of  10 | elapsed:    1.2s finished


IndexError: index 10 is out of bounds for axis 0 with size 10